# GWP* Analysis

_Dataset_: Supplementary data for Megill et al. (2024): "Alternative climate metrics to the Global Warming Potential are more suitable for assessing aviation non-CO2 effects"

_Authors_:

- Liam Megill (1, 2), https://orcid.org/0000-0002-4199-6962   
- Kathrin Deck (2)  
- Volker Grewe (1, 2), https://orcid.org/0000-0002-8012-6783  

_Affiliation (1)_: Deutsches Zentrum für Luft- und Raumfahrt (DLR), Institut für Physik der Atmosphäre, Oberpfaffenhofen, Germany

_Affiliation (2)_: Delft University of Technology (TU Delft), Faculty of Aerospace Engineering, Section Aircraft Noise and Climate Effects (ANCE), Delft, The Netherlands

_Corresponding author_: Liam Megill, liam.megill@dlr.de

_doi_: https://doi.org/10.1038/s43247-024-01423-6

---


### Summary
This notebook creates Figure 3 of the linked paper, showing the CO2eq emissions calculated using the GWP* climate metric over time, demonstrating the flow-based nature of the GWP*.

### Linked data
- `MVMC/batch_1/Fleet1`: Fleet1 of the multivariate fleet analysis used as an example fleet in this notebook
- `MVMC/E_bg_fuel_MC.txt`: Fleet emission input file to AirClim, originally from the multivariate fleet analysis and used here as an example

---

### Copyright

Copyright © 2024 Liam Megill

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0. Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

## Setup

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

colors = ['#2271B2', '#3DB7E9', '#F748A5', '#359B73', '#D55E00', '#E69F00', '#F0E442']
sns.set_context("paper", rc={"lines.linewidth": 2})
sns.set_style("ticks")
labels = ["Total", r"CO$_2$", r"H$_2$O", r"O$_3$", r"CH$_4$", "Contrails", "PMO"]

## Function definition

In [ ]:
def load_rf_dt(directory):
    """Load the radiative forcing (RF) and temperature change (dT) responses of a given fleet.

    Args:
        directory (_str_): Path to data directory.

    Returns:
        rf_arr (_np.ndarray_): Array (7, len(years)) of radiative forcing results for each emission species
        dt_arr (_np.ndarray_): Array (7, len(years)) of temperature change results for each emission species
        years (_np.ndarray_): Array of response years from AirClim
    """
    emission_species = ["CO2", "H2O", "O3", "CH4", "Cont", "PMO"]
    years = np.loadtxt(directory + "RF_CO2_taumean_rfmean.txt", skiprows=2, usecols=0)
    rf_arr = np.zeros((7, len(years)))  # define empty RF array
    dt_arr = np.zeros((7, len(years)))  # define empty dT array
    # populate rf_arr and dt_arr
    for i in range(len(emission_species)):
        rf_name = "RF_" + emission_species[i] + "_taumean_rfmean.txt"
        dt_name = "dT_" + emission_species[i] + "_taumean_rfmean_lammean.txt"
        rf_lst = np.loadtxt(directory + rf_name, skiprows=2, usecols=1)  # uses default AirClim output format
        dt_lst = np.loadtxt(directory + dt_name, skiprows=2, usecols=1)  # uses default AirClim output format
        rf_arr[i + 1, :] = rf_lst  # i+1 because i=0 is the total
        dt_arr[i + 1, :] = dt_lst
    # Add totals
    rf_arr[0, :] = np.sum(rf_arr[:, :], axis=0)
    dt_arr[0, :] = np.sum(dt_arr[:, :], axis=0)
    return rf_arr, dt_arr, years


def gwps_co2eq(H, forcing_profiles, temperature_profiles, years, CO2_emissions,
               agwph_co2_file="../data/MVMC/AGWPH_CO2_SSP2-45.txt"):
    """Calculate CO2eq emissions using GWP*_H

    Args:
        H (_float_): Time horizon [years]
        forcing_profiles (_np.ndarray_): RF values from AirClim
        temperature_profiles (_np.ndarray_): dT values from AirClim
        years (_np.ndarray_): Array of response years from AirClim
        CO2_emissions (_np.ndarray_): Array of yearly CO2 emissions
        agwph_co2_file (_str_): Path to AGWP_CO2(H) file (AGWP as a function of H for CO2 for 1<=H<=100)

    Returns:
        gwps_co2eq (_np.ndarray_): Array of CO2eq emissions 
        yr_idx1 (_np.ndarray_): Array of corresponding indicies in years
    """
    dt = 20
    s = 0.25  # in line with Smith et al. (2021) for CH4
    agwph_co2 = np.loadtxt(agwph_co2_file)
    yr_idx1 = np.arange(dt, len(forcing_profiles[0, :]))
    agwp_co2 = agwph_co2[H - 1] * 1e9
    dFdt = np.subtract(forcing_profiles[:, np.arange(dt, len(years))],
                       forcing_profiles[:, np.arange(0, len(years) - dt)]) / dt / 1000.
    F_av = np.array([[sum(forcing_profiles[j, i - dt + 1: i + 1]) / dt
                      for i in np.arange(dt, len(years))]
                     for j in range(7)]) / 1000.
    g = (1 - np.exp(-s / (1 - s))) / s  # Extension by Smith et al. (2021)
    gwps_co2eq = g * ((1 - s) * dFdt * H / agwp_co2 + s * F_av / agwp_co2)
    gwps_co2eq[1, :] = CO2_emissions[yr_idx1]  # Add co2 emissions directly
    gwps_co2eq[0, :] = np.sum(gwps_co2eq[1:, :], axis=0)
    return gwps_co2eq, yr_idx1

## Plot GWP* as a function of time

This figure corresponds to Figure 3 in the linked paper. Uncomment the last line to save the figure to the `images` folder.

In [ ]:
# initialise figure
fig, ax = plt.subplots(figsize=(7, 3.5))

# plot emission profile in inset axes
ref_fuel_years = np.loadtxt("../data/MVMC/E_bg_fuel_MC.txt", skiprows=3, usecols=0)
emission_profile = np.loadtxt("../data/MVMC/E_bg_fuel_MC.txt", skiprows=3, usecols=2)
insax = ax.inset_axes([0.6, 0.7, 0.35, 0.25])
insax.plot(ref_fuel_years, emission_profile * 14.498, "k")
insax.set_ylabel('Fuel use [Tg]')
insax.set_xlim([2020, 2120])

# plot GWP* response
rf, dt, years = load_rf_dt("../data/MVMC/batch_1/Fleet1/")
co2_emis = (np.loadtxt("../data/MVMC/batch_1/Fleet1/CO2_emis.txt"))[:, 1]
gwps_response, gwps_idx = gwps_co2eq(100, rf, dt, years, co2_emis)
for i in range(7):
    ax.plot(years[gwps_idx], gwps_response[i, :], color=colors[i], label=labels[i])

ax.set_ylabel('CO$_2$-eq using GWP* [Tg]', fontsize=12)
ax.legend(loc='lower left', ncol=2, fontsize=10)
ax.set_xlim([2020, 2120])
ax.set_xlabel('Year', fontsize=12)
ax.tick_params(axis='x', labelsize=12)
ax.tick_params(axis='y', labelsize=12)
fig.tight_layout()

# save figure
fig_savename = "../images/gwps_example.png"
# fig.savefig(fig_savename, dpi=600)